# MnMS : Creation of a Public Transport line

# 1. From scratch

## 1. The Mobility Services

### Add a "BUS" Mobility Service

The name of the "BUS" Mobility Service will be **'B0'**.

In [ ]:
from mnms.mobility_service.public_transport import PublicTransportMobilityService

# register_stop(sid, lid, relative_position)
# sid: id of the stop created you want to give
# lid: id of the link where the stop should be
# relative_position: position on the link, should be between 0 and 1

# West to East bus stops
roads.register_stop('S_WE_0', 'WEST_2_2', 0.5)
roads.register_stop('S_WE_1', '2_7', 0.5)
roads.register_stop('S_WE_2', '7_12', 0.5)
roads.register_stop('S_WE_3', '12_17', 0.5)
roads.register_stop('S_WE_4', '17_22', 0.5)
roads.register_stop('S_WE_5', '22_EAST_2', 0.5)

# North to South bus stops
roads.register_stop('S_NS_0', 'NORTH_2_14', 0.5)
roads.register_stop('S_NS_1', '14_13', 0.5)
roads.register_stop('S_NS_2', '13_12', 0.5)
roads.register_stop('S_NS_3', '12_11', 0.5)
roads.register_stop('S_NS_4', '11_10', 0.5)
roads.register_stop('S_NS_5', '10_SOUTH_2', 0.5)

bus_service = PublicTransportMobilityService('B0')


### Create the "BUS" Layer

The **'BUS'** Layer will contain the "B0" mobility service.


In [ ]:
from mnms.graph.layers import PublicTransportLayer
from mnms.vehicles.veh_type import Bus
from mnms.time import TimeTable, Dt

# default_speed: 30
bus_layer = PublicTransportLayer(roads, 'BUS', Bus, 30, services=[bus_service])

# create_line(lid, stops, sections, timetable)
# lid: id of the line created you want to give
# stops: list containing the stops of the line
# sections: list containing sublists where each sublist contains two links, the list is used to connect the stops
#           between them. Please see example below.
# timetable: timetable of the line

# create_table_freq(start, stop, dt)
# start: start time (first departure)
# end: end time (last departure)
# dt: frequency, unit (minutes/seconds) can be specified

# Central bus line West to East
bus_layer.create_line('L_WE',
                    ['S_WE_0', 'S_WE_1', 'S_WE_2', 'S_WE_3', 'S_WE_4', 'S_WE_5'],
                    [['WEST_2_2', '2_7'], ['2_7','7_12'], ['7_12', '12_17'], ['12_17', '17_22'], ['17_22', '22_EAST_2']],
                    TimeTable.create_table_freq('08:00:00', '12:00:00', Dt(minutes=1)))

# Central bus line North to South
bus_layer.create_line('L_NS',
                    ['S_NS_0', 'S_NS_1', 'S_NS_2', 'S_NS_3', 'S_NS_4', 'S_NS_5'],
                    [['NORTH_2_14', '14_13'], ['14_13', '13_12'], ['13_12', '12_11'], ['12_11', '11_10'], ['11_10', '10_SOUTH_2']],
                    TimeTable.create_table_freq('08:00:00', '12:00:00', Dt(minutes=1)))

## 2. The Multilayer graph

### Create an OD Layer

In [ ]:
from mnms.generation.layers import _generate_matching_origin_destination_layer
odlayer = _generate_matching_origin_destination_layer(roads)

### Create a Multi Layer Graph

The **"Transit Link"** is used to connect a layer to another one.

In [ ]:
from mnms.graph.layers import MultiLayerGraph

# MultiLayerGraph(layers, odlayer, connection_distance)
mlgraph = MultiLayerGraph([car_layer, bus_layer], odlayer, 1e-5)

# connect_layers(lid, upstream, downstream, length, costs)
# lid: id of the layer connection (or transit link) created you want to give
# upstream: id of node from upstream layer
# downstream: id of node from downstream layer
# length: length of the transit link
# costs: dict with the costs of the transit link

mlgraph.connect_layers("TL1", "CAR_WEST_2", "L_WE_S_WE_0", 5, {})
mlgraph.connect_layers("TL2", "CAR_2", "L_WE_S_WE_0", 5, {})

mlgraph.connect_layers("TL3", "CAR_22", "L_WE_S_WE_5", 5, {})
mlgraph.connect_layers("TL4", "CAR_EAST_2", "L_WE_S_WE_5", 5, {})

mlgraph.connect_layers("TL5", "CAR_NORTH_2", "L_NS_S_NS_0", 5, {})
mlgraph.connect_layers("TL6", "CAR_14", "L_NS_S_NS_0", 5, {})

mlgraph.connect_layers("TL7", "CAR_10", "L_NS_S_NS_5", 5, {})
mlgraph.connect_layers("TL8", "CAR_SOUTH_2", "L_NS_S_NS_5", 5, {})

mlgraph.connect_layers("TL9", "CAR_12", "L_WE_S_WE_2", 5, {})
mlgraph.connect_layers("TL10", "CAR_12", "L_WE_S_WE_3", 5, {})
mlgraph.connect_layers("TL11", "CAR_12", "L_NS_S_NS_2", 5, {})
mlgraph.connect_layers("TL12", "CAR_12", "L_NS_S_NS_3", 5, {})

# 2. From an existing network

## 1. Global Structure (recap)

Below is a recap of the global structure of the JSON Network file, for more information please check out the notebook TutorialJsonNetwork.

The Json network file contains two main parts:

- The physical/geometrical part, ROADS, which contains a list of nodes NODES, a list of stops STOPS, a list of sections SECTIONS and a list of zones ZONES

- The topological part, LAYERS, which contains the different mobility layers (CAR, BUS, TRAM, METRO etc). We can distinguish two different structures, one for the mobility service CAR and another one for the mobility service PT (Public Transportation) :
    - The LAYER CAR
    - The LAYER PT
    
- The general structure is presented below :

In [ ]:
{
    "ROADS": {
        "NODES": {},
        "STOPS": {},
        "SECTIONS": {},
        "ZONES": {}
    },
    
    "LAYERS": [
        {},
        {},
        {},
        {}
    ]
    
}

## 2. Amsterdam example

### 1. Add the Nodes

Add the Nodes in "ROADS" > "NODES"

In [ ]:
{
    "Tram_exit_1": {
        "id": "Tram_exit_1",
        "position": [
            621014.0199326351,
            5801555.23821623
        ]
    },
    
    "Tram_entry_1": {
        "id": "Tram_entry_1",
        "position": [
            631637.3340760886,
            5802779.135158912
        ]
    },
}

### 2. Add the Stops

Add the Stops in "ROADS" > "STOPS"

In [ ]:
{
    "Matterhorn": {
        "id": "Matterhorn",
        "section": "Tram_1_OE",
        "relative_position": 0.99,
        "absolute_position": [
         621014.0199326351,
         5801555.23821623
        ]
    },

    "Matterhorn2": {
        "id": "Matterhorn2",
        "section": "Tram_1_EO",
        "relative_position": 0.01,
        "absolute_position": [
         621014.0199326351,
         5801755.23821623
        ]
    },

    "Muiderpoorstation": {
        "id": "Muiderpoorstation",
        "section": "Tram_1_OE",
        "relative_position": 0.01,
        "absolute_position": [
         631637.3340760886,
         5802779.135158912
        ]
    },

    "Muiderpoorstation2": {
        "id": "Muiderpoorstation2",
        "section": "Tram_1_EO",
        "relative_position": 0.99,
        "absolute_position": [
         631637.3340760886,
         5802979.135158912
        ]
    },
}

### 3. Add the Sections

Add the Sections in "ROADS" > "SECTIONS"

In [ ]:
{
    "Tram_1_EO": {
        "id": "Tram_1_EO",
        "upstream": "Tram_exit_1",
        "downstream": "Tram_entry_1",
        "length": 12200.00,
        "zone": "RES"
    },
    
    "Tram_1_OE": {
        "id": "Tram_1_OE",
        "upstream": "Tram_entry_1",
        "downstream": "Tram_exit_1",
        "length": 12200.00,
        "zone": "RES"
    },
}

### 4. Add the Zones

Add the Sections ID in "ROADS" > "ZONES" > "RES" > "sections" where "RES" is the Reservoir where the Sections/Links of your Public Transport should be.

In [ ]:
{
    "ZONES": {
        "RES": {
            "id": "RES",
            "sections": [
                "Tram_1_EO",
                "Tram_1_OE",
            ]
        }
    }
}

### 5. Add the Layer

In [ ]:
{
    "ID": "TRAMLayer",
    "TYPE": "mnms.graph.layers.PublicTransportLayer",
    "VEH_TYPE": "mnms.vehicles.veh_type.Tram",
    "DEFAULT_SPEED": 40.0,
    "SERVICES": [],
    "LINES": [
     {
          "ID": "1_EO",
          "STOPS": [
               "Matterhorn2",
               "Muiderpoorstation2"
          ],
          "SECTIONS": [
           [
                "Tram_1_EO"
           ]
          ],
         "TIMETABLE": [
               "05:00:00.00",
               "05:03:00.00",
               "05:06:00.00",
               "05:09:00.00",
               "05:12:00.00",
                "..."
         ],
         "BIDIRECTIONAL": false
     },
    {
          "ID": "1_OE",
          "STOPS": [
                "Muiderpoorstation",
                "Matterhorn"
          ],
          "SECTIONS": [
           [
                "Tram_1_OE"
           ]
          ],
         "TIMETABLE": [
               "05:00:00.00",
               "05:03:00.00",
               "05:06:00.00",
               "05:09:00.00",
               "05:12:00.00",
                "..."
         ],
         "BIDIRECTIONAL": false
     },
    
}